In [7]:
import pandas as pd

aug_information = pd.read_csv(f"result/aug/wikitable_test_summary.csv", index_col='table_id')


In [8]:
aug_information[']

,summary,column_description
table_id,,
csv/203-csv/840.tsv,List of Ambassadors of Russia to Austria,Name<The name of the ambassador>\nTitle<The ti...
csv/204-csv/65.tsv,This table shows the results of the Men's 50 m...,Rank<The ranking of the swimmer in the event> ...
csv/203-csv/275.tsv,This table shows the results of the 1986 Austr...,Pos<Position of the driver in the race>
csv/204-csv/272.tsv,The table shows the results of Sky Track Cycli...,Date<The date of the competition>\nCompetition...
csv/203-csv/328.tsv,This table provides information about episodes...,num<The episode number>\nEpisode<The title of ...


In [13]:
from dateutil import parser
parser.parse('1931')

datetime.datetime(1931, 4, 29, 0, 0)

In [28]:
import pandas as pd
composition_information = pd.read_csv(f"result/aug/sqa_test_summary_alone.csv", index_col='table_id')
len(composition_information)

212

In [14]:
test_string = 'year<The year the coin was released>\ntheme<The theme of the coin>\nartist<The artist responsible for the design of the coin>\nmintage<The number of coins minted>\nissue_price<The price at which the coin was issued>'
import re 
test = test_string.split('\n')
print(test)
pattern=r'([^<]*)<([^>]*)>'
matches = re.finditer(pattern, test_string)
for match in matches:
   print(match.group(2))

['year<The year the coin was released>', 'theme<The theme of the coin>', 'artist<The artist responsible for the design of the coin>', 'mintage<The number of coins minted>', 'issue_price<The price at which the coin was issued>']
The year the coin was released
The theme of the coin
The artist responsible for the design of the coin
The number of coins minted
The price at which the coin was issued


## Start


In [1]:
# transform data into sqlite database
from data_loader import TableLoader

table_loader = TableLoader(table_name='wikitable', split='test', use_sample=True, small_test=False)



In [3]:
len(table_loader.dataset)

300

In [11]:
from datasets import load_dataset
totto = load_dataset('./data_loader/totto_zh.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")
tabfact = load_dataset('./data_loader/tabfact.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")
sqa = load_dataset('./data_loader/sqa.py', verification_mode="no_checks", cache_dir="/media/disk2/datasets")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [34]:
filtered_dataset = table_loader.dataset.filter(lambda example: example['small_test'])

Filter: 100%|██████████| 1000/1000 [00:00<00:00, 4669.79 examples/s]


In [35]:
filtered_dataset

Dataset({
    features: ['id', 'table', 'statement', 'label', 'hardness', 'small_test'],
    num_rows: 147
})

In [13]:
len(table_loader.dataset)


47

In [35]:
test_sample = table_loader.dataset[2]
test_sample

{'id': 996,
 'table': {'id': '2-16369528-1.html.csv',
  'header': ['year',
   'best teenage / young adult',
   "reader 's vote",
   'best non - fiction',
   'lifetime achievement'],
  'rows': [['1996', 'na', 'na', 'na', 'jon cleary'],
   ['1997',
    'na',
    'na',
    'how to write crime edited by marele day',
    'alan yates (aka carter brown )'],
   ['1998', 'na', 'na', 'na', 'na'],
   ['1999', 'na', 'na', 'na', 'peter corris'],
   ['2000', 'na', 'na', 'na', 'na'],
   ['2001',
    'na',
    'bleeding hearts by lindy cameron',
    'na',
    'professor stephen knight'],
   ['2002',
    'blue murder by ken catran',
    'apartment 255 by bunty avieson',
    'na',
    'patrick gallagher'],
   ['2003', 'na', 'na', 'na', 'kerry greenwood'],
   ['2004', 'na', 'na', 'na', 'bob bottom'],
   ['2005', 'na', 'na', 'na', 'stuart coupe'],
   ['2006', 'na', 'na', 'na', 'andrew rule and john silvester'],
   ['2007', 'na', 'na', 'na', 'sandra harvey and lindsay simpson'],
   ['2008', 'na', 'na', 'na

### Data Augmentation

In [ ]:
#引入 schema(table measurement) statistical(numerical) enum string(char) date
#引入 term explanation（table comment(RAG)）
#引入 column summarization
#是否需要search engine
#table size, statistical features, header hierarchy


In [50]:
#schema 
from data_loader import TableFormat
from langchain.prompts.prompt import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.tech/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
# formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
pre_instruction = PromptTemplate(input_variables=["table"], template=
"""
Instruction: Given the following table, you will add Metadata about the columns in the table.
Metadata includes:
- Numerical: whether the column content is numeric type like int or float.
- Char: whether the column content is a text or description.
- Date: whether the column content is datetime.

You need to output all the column names with metadata in angle brackets.
Example: name<Char>, launched<Date>, count<Numerical>

Table: {table}
Output:
""")
# 
# output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html()))])

In [16]:
output.response_metadata['token_usage']

{'completion_tokens': 1, 'prompt_tokens': 269, 'total_tokens': 270}

In [51]:
#summarization
pre_instruction_summary = PromptTemplate(input_variables=['table'], template=
"""
Instruction: Given the following table, you need to first summarize the contents of the table, then based on the summay, give a concluded description to each of the column.
Table: {table}

The output should use the following format: 
table summary: #summary for table contents
column description: You need to output all the column names with description in angle brackets
example: launched<The launched date for the competition> date<The date of the match>
""")
output = model.invoke([HumanMessage(content=pre_instruction_summary.format(table=formatter.format_html()))])

In [52]:
print(output.content)

table summary: The table shows the performance of a player in different tennis tournaments from 1986 to 1999.

column description: 
tournament<The name of the tennis tournament>
1986<The player's performance in the tournament in 1986>
1988<The player's performance in the tournament in 1988>
1989<The player's performance in the tournament in 1989>
1990<The player's performance in the tournament in 1990>
1991<The player's performance in the tournament in 1991>
1992<The player's performance in the tournament in 1992>
1993<The player's performance in the tournament in 1993>
1994<The player's performance in the tournament in 1994>
1995<The player's performance in the tournament in 1995>
1996<The player's performance in the tournament in 1996>
1997<The player's performance in the tournament in 1997>
1998<The player's performance in the tournament in 1998>
1999<The player's performance in the tournament in 1999>


In [11]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = "true"

In [10]:
#table_size
def get_table_size():
    return f'The table has {formatter.data.shape[0]} rows and {formatter.data.shape[1]} columns.'
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
from data_loader import TableAug,TableFormat
aug = TableAug(model)
# aug.schema_aug(test_sample)
test_sample = table_loader.dataset[0]
formatter = TableFormat(format='none')
output = aug.batch_summary_aug(formatter, table_loader.dataset[:2], batch_size=2, output_token=True)
# batch_data = table_loader.dataset[:2]

# print([formatter.load_data_from_dic({key: value[i] for key, value in batch_data.items()}).format_html() for i in range(len(batch_data.keys())) ])
# print([formatter.load_data_from_dic({key: value[i] for key, value in table_loader.dataset[:2].items()}).format_html() for i in range(2)])

KeyError: 0

In [9]:
print(output[0])

{'text': 'table summary: The table provides information about different wind farms including their names, scheduled dates, capacity in megawatts, number of turbines, types of turbines, and locations.\n\ncolumn description: \nwind farm<The name of the wind farm>\nscheduled<The scheduled date for the wind farm>\ncapacity (mw)<The capacity of the wind farm in megawatts>\nturbines<The number of turbines at the wind farm>\ntype<The type of turbines used at the wind farm>\nlocation<The location of the wind farm>'}


In [12]:
formatter = TableFormat(format='none',data=test_sample)
aug.summary_aug(formatter)

("Golf tournament winners' information",
 "player<The name of the player>\ncountry<The country the player represents>\nyear (s) won<The year(s) the player won the tournament>\ntotal<The total score of the player>\nto par<The score in relation to par (under par or over par)>\nfinish<The player's finishing position in the tournament>")

### Few Shot learning

In [9]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
summary_examples = ['The columns in the table are "wind farm, scheduled, capacity (mw), turbines, type, and location." The rows in the table represent different wind farms, with information about their scheduled dates, capacity, number of turbines, type, and location.',
                    "The table provides information about different destroyers, including their kanji (Japanese characters), names, builders, dates when they were laid down, launched, and completed."]
subtable_examples = ['Columns(wind farm, scheduled, turbines), Rows(12)',
                    'Columns(kanji, builder), Rows(ALL)']
inds = [0, 100]
examples_prompt = PromptTemplate(input_variables=["table", "claim", "summary", "subtable"], template=
"""
Table: {table}
Claim: {claim}
Summary: {summary}
Subtable: {subtable}""")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
num_k = 2
# examples = [examples_prompt.format(**{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]]).format_html(),
#                                     "claim": table_loader.dataset[inds[i]]['statement'],
#                                     "summary": summary_examples[i],
#                                     "subtable": subtable_examples[i]}) for i in range(num_k)]

examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(),
                                    "claim": table_loader.dataset[inds[i]]['statement'],
                                    "summary": summary_examples[i],
                                    "subtable": subtable_examples[i]} for i in range(num_k)]
prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="Instruction: Given the following table and claim, let's first summarize the contents of the rows and columns of the table, and then select relevent rows/columns in the given table that support or oppose the statement.",
    suffix=
    """
Table: {table}
Claim: {claim}
    """,
    input_variables=["table", "claim"],
)
print(prompt.format(table=formatter.format_html(), claim=test_sample['statement']))

Instruction: Given the following table and claim, let's first summarize the contents of the rows and columns of the table, and then select relevent rows/columns in the given table that support or oppose the statement.


Table: <table>
<thead>
<tr><th>  wind farm</th><th>  scheduled</th><th>  capacity (mw)</th><th>  turbines</th><th>              type</th><th>      location</th></tr>
</thead>
<tbody>
<tr><td>codling    </td><td>unknown    </td><td>1100           </td><td>220       </td><td>unknown           </td><td>county wicklow</td></tr>
<tr><td>carrowleagh</td><td>2012       </td><td>36.8           </td><td>16        </td><td>enercon e - 70 2.3</td><td>county cork   </td></tr>
<tr><td>gortahile  </td><td>2010 autumn</td><td>20             </td><td>8         </td><td>nordex n90        </td><td>county laois  </td></tr>
</tbody>
</table>
Claim: the scheduled date for the farm with 17 turbine be 2012
Summary: The columns in the table are "wind farm, scheduled, capacity (mw), turbines, t

In [5]:
from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])

AIMessage(content='Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(2)', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 931, 'total_tokens': 972}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None})

### Few-shot for operations

In [38]:
from data_loader import TableFormat
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
inds = [0,2]
summary_examples = ['The columns in the table are "wind farm, scheduled, capacity (mw), turbines, type, and location." The rows in the table represent different wind farms, with information about their scheduled dates, capacity, number of turbines, type, and location.',
                    "The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues."]
Output_examples = ['wind farm<DELETE>, scheduled<KEEP>, capacity (mw)<DELETE>, turbines<KEEP>, type<KEEP>, location<KEEP>', 'event name<KEEP> established<KEEP> category<KEEP> sub category<DELETE> main venue<DELETE>']
examples_prompt = PromptTemplate(input_variables=["table", "claim", "output"], template=
"""
Table: {table}
Claim: {claim}
Output: {output}""")
num_k = 2
examples_dict = [{"table": TableFormat(format='none', data=table_loader.dataset[inds[i]], use_sampling=True).format_html(),
                                    "claim": table_loader.dataset[inds[i]]['statement'],
                                    # "summary": summary_examples[i],
                                    "output": Output_examples[i]} for i in range(num_k)]
prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Instruction: Given the following table and claim, you will output the operations corresponding to each column which can help us judging the truth or falsity of claim.
Operations: DELETE(delete column unrelevant to the claim), KEEP(keep column relevant to the claim), GROUP BY(combine aggregate functions and group the result set by one or more columns), COUNT(returns the number of rows in column), AVG(returns the average value of a numeric column), SUM(returns the sum of a numeric column), MAX(returns the max value of a numeric column), MIN(returns the min value of a numeric column), ORDER BY(sort the value in ascending order)""",
    suffix=
    """
Table: {table}
Thought: your reason here
Claim: {claim}
    """,
    input_variables=["table", "claim"],
)
print(prompt.format(table=formatter.format_html(), claim=test_sample['statement']))

# output = model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])



You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
Instruction: Given the following table and claim, you will output the operations corresponding to each column which can help us judging the truth or falsity of claim.
Operations: DELETE(delete column unrelevant to the claim), KEEP(keep column relevant to the claim), GROUP BY(combine aggregate functions and group the result set by one or more columns), COUNT(returns the number of rows in column), AVG(returns the average value of a numeric column), SUM(returns the sum of a numeric column), MAX(returns the max value of a numeric column), MIN(returns the min value of a numeric column), ORDER BY(sort the value in ascending order)


Table: <table>
<thead>
<tr><th>  wind farm</th><th>  scheduled</th><th>  capacity (mw)</th><th>  turbines</th><th>              type</th><th>      location</th></tr>
<

In [39]:
output = model.invoke([HumanMessage(content=prompt.format(table=formatter.format_html(), claim=test_sample['statement']))])
print(output)

content='Output: date<DELETE>, tournament<DELETE>, surface<DELETE>, partner<KEEP>, opponents<KEEP>, score<DELETE>' response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 1037, 'total_tokens': 1063}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}


In [35]:
print(formatter.format_psql())
print(test_sample['statement'])
print(test_sample['label'])

+------------------+-----------------------------------+-----------+---------------------+------------------------------------------------+-----------------------+
| date             | tournament                        | surface   | partner             | opponents                                      | score                 |
|------------------+-----------------------------------+-----------+---------------------+------------------------------------------------+-----------------------|
| 15 january 2007  | algiers 2 , algeria               | clay      | rushmi chakravarthi | barbora matusova anna savitskaya               | 6 - 2 , 6 - 0         |
| 11 february 2008 | mallorca 2 , spain                | clay      | stephanie vogt      | leticia costas - moreira maite gabarrus alonso | 7 - 6 (7 - 2) , 6 - 3 |
| 28 april 2008    | makarska , croatia                | clay      | stephanie vogt      | tadeja majerić maša zec peškirič               | 7 - 5 , 6 - 2         |
| 8 september 20

## Zero-shot learning


In [22]:
from tabulate import tabulate
formatter = TableFormat(format='none', data=test_sample)
print(test_sample['statement'])
print(tabulate(formatter.data, headers=formatter.data.columns, tablefmt='psql'))
# print('Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(2)')

uraga dock company produce the most destroyer on the list , at 3
+----+---------+-------------------+--------------------------------+------------------+------------------+-------------------+
|    | kanji   | name              | builder                        | laid down        | launched         | completed         |
|----+---------+-------------------+--------------------------------+------------------+------------------+-------------------|
|  0 | 睦月    | mutsuki dd - 19   | sasebo naval arsenal , japan   | 21 may 1924      | 23 july 1925     | 25 march 1926     |
|  1 | 如月    | kisaragi dd - 21  | maizuru naval arsenal , japan  | 3 june 1924      | 5 june 1925      | 21 december 1925  |
|  2 | 彌生    | yayoi dd - 23     | uraga dock company , japan     | 11 january 1924  | 11 july 1925     | 28 august 1926    |
|  3 | 卯月    | uzuki dd - 25     | ishikawajima shipyards , japan | 11 january 1924  | 15 october 1925  | 14 september 1926 |
|  4 | 皐月    | satsuki dd - 27   | fujinagata s

In [10]:
from data_loader import TableFormat, TableLoader
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
table_loader = TableLoader(table_name='wikitable', split='test', use_sample=True, small_test=False)
test_sample = table_loader.dataset[40]
formatter = TableFormat(format='none', data=test_sample, use_sampling=True)
pre_instruction_schema = PromptTemplate(input_variables=["table"], template="""
Instruction: Given the following table, you need to summarize the contents of the table and tell what table is about.
Table: {table}

The output should use the following format: 
Summary: #summary for table contents

Summary:
""")
output = model.invoke([HumanMessage(content=pre_instruction_schema.format(table=formatter.format_html(),))])



In [11]:
print(output.content)

The table provides information about different games played by a football team. It includes details like the week number, date of the game, opponent team, game result, game site, NFL recap, and the attendance for each game.


In [40]:
from tabulate import tabulate
print(tabulate(formatter.data, headers=formatter.data.columns, tablefmt='psql'))
print('Summary: The table provides information about different events, including their names, establishment years, categories, subcategories, and main venues.\nSubtable: Columns(event name, established, category), Rows(touchdown atlantic)')

+----+--------------------------------------------+---------------+----------------+----------------+--------------------------+
|    | event name                                 |   established | category       | sub category   | main venue               |
|----+--------------------------------------------+---------------+----------------+----------------+--------------------------|
|  0 | dieppe kite international                  |          2001 | sporting       | kite flying    | dover park               |
|  1 | the frye festival                          |          2000 | arts           | literary       | university of moncton    |
|  2 | hubcap comedy festival                     |          2000 | arts           | comedy         | various                  |
|  3 | touchdown atlantic                         |          2010 | sporting       | football       | moncton stadium          |
|  4 | atlantic nationals automotive extravaganza |          2000 | transportation | automotive  

In [41]:
print(test_sample['statement'])

touchdown atlantic , in the category of sporting , be establish in 2010


In [38]:
import os
from tenacity import retry, stop_after_attempt, wait_random_exponential
from typing import List
from openai import OpenAI
class Config:
    def __init__(self) -> None:
        self.openai_api_key = "sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW"
        self.base_url = "https://api.chatanywhere.cn/v1"
        self.model = "gpt-3.5-turbo"

class CallLLM:
    """Class for calling the OpenAI Language Model API."""

    def __init__(self, config: Config):
        self.client = OpenAI(api_key=config.openai_api_key,
                             base_url=config.base_url)
        self.model = config.model
        
    @retry(wait=wait_random_exponential(min=30, max=60), stop=stop_after_attempt(1000))
    def generate_text(self, prompt: List[str]) -> List[str]:
        """Generate text based on the prompt and instruction."""

            # batched examples, with xx completions per request
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}], 
            temperature=0,
            max_tokens=96,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0,
        )
        # match completions to prompts by index
        return response.choices[0].message.content.strip()
configs = Config()  
llm = CallLLM(config=configs)


In [39]:

input_text = "\n".join(input)
llm.generate_text(input_text)

'1'

In [15]:
import pandas as pd
def process_input(sample, instruction):
    df = pd.DataFrame(columns=sample["table"]["header"])
    for i, line in enumerate(sample['table']['rows']):
        df.loc[i] = line
        
    texts = [instruction,
          "The database table DF is shown as follows: \n",
          df.to_html(),
          "query:",
          sample['statement'],
           "Output the code braced by '```'. \n SQL:"]
    
    label = sample['label']
    return "\n".join(texts), label 
    

In [4]:
subtable_instruction = """
You are a SQLite expert. Given an input query, identify critical values and ranges of the table, then create a syntactically correct SQLite query to create a VIEW. To create a syntactically correct SQL view, the selected data within this view must be helpful in answering the question. During the construction of the view, if column names are confusing, rename the columns accordingly.
"""

In [16]:
print(process_input(test_sample, subtable_instruction))

('\nYou are a SQLite expert. Given an input query, identify critical values and ranges of the table, then create a syntactically correct SQLite query to create a VIEW. To create a syntactically correct SQL view, the selected data within this view must be helpful in answering the question. During the construction of the view, if column names are confusing, rename the columns accordingly.\n\nThe database table DF is shown as follows: \n\n<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>date</th>\n      <th>result</th>\n      <th>score</th>\n      <th>brazil scorers</th>\n      <th>competition</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>may 11 , 1919</td>\n      <td>w</td>\n      <td>6 - 0</td>\n      <td>friedenreich (3) , neco (2) , haroldo</td>\n      <td>south american championship</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>may 18 , 1919</td>\n      <td>w</td>\n      <td>3 - 1

### Simple pipeline

In [6]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db')
normalized = table_loader.normalize_table(test_sample)
df = pd.DataFrame(columns=normalized['table']['header'])
for ind, r in enumerate(normalized['table']['rows']):
    df.loc[ind] = r
# print(' '.join(normalized['table']['header']) + '*************')
df.to_sql(name='ind100', con=engine, if_exists='replace', index=False)
# table_loader.table2db(engine, test_sample)

12

In [34]:
from tabulate import tabulate
structured_data_markdown = tabulate(
            df, headers=df.columns, tablefmt="pipe", showindex=True
        )

In [35]:
structured_data_markdown

'|    | date          | result   | score   | brazil scorers                        | competition                 |\n|---:|:--------------|:---------|:--------|:--------------------------------------|:----------------------------|\n|  0 | may 11 , 1919 | w        | 6 - 0   | friedenreich (3) , neco (2) , haroldo | south american championship |\n|  1 | may 18 , 1919 | w        | 3 - 1   | heitor , amílcar , millon             | south american championship |\n|  2 | may 26 , 1919 | d        | 2 - 2   | neco (2)                              | south american championship |\n|  3 | may 29 , 1919 | w        | 1 - 0   | friedenreich                          | south american championship |\n|  4 | june 1 , 1919 | d        | 3 - 3   | haroldo , arlindo (2)                 | taça roberto cherry         |'

In [1]:

from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
model = ChatOpenAI(model_name="gpt-4", openai_api_base="https://api.chatanywhere.cn/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")

In [7]:
import re
def extract_yes_no_and_map(text):
    # Convert the input text to lowercase for case-insensitive matching
    text = text.lower()

    # Define regular expressions for yes/no matching
    yes_patterns = [r'\byes\b', r'\btrue\b']
    no_patterns = [r'\bno\b', r'\bfalse\b']

    # Check for "0"
    if text == "0":
        return "0"

    # Check for "1"
    if text == "1":
        return "1"

    # Check for yes
    for pattern in yes_patterns:
        if re.search(pattern, text):
            return "1"

    # Check for no
    for pattern in no_patterns:
        if re.search(pattern, text):
            return "0"

    # Return 2 if neither yes nor no is found
    return "2"

def eval_fv_match(pred_list, gold_list):
        acc = 0.0
        for pred, gold in zip(pred_list, gold_list):
            pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
            if pred == gold:
                acc += 1
        acc = acc / len(pred_list)
        return acc
    
def process_input(sample, instruction):
    df = pd.DataFrame(columns=sample["table"]["header"])
    for i, line in enumerate(sample['table']['rows']):
        df.loc[i] = line
    texts = [instruction,
          "the table needed to be answered: \n",
          df.to_html(),
          "query:",
          sample['statement'],
           "answer is: \n"]
    
    label = sample['label']
    return "\n".join(texts), label 

In [37]:
#简易版本的pipeline
instruction = "Read the table below to verify whether the provided claim/query are true or false. Return 0 if it's false, or 1 if it's true. Only return 0 or 1 without any other information. \n"
num_samples = 32
batch_size = 32
num_batches = num_samples // batch_size
batches = []
for batch_num in range(num_batches):
    batch_prompt, ground = [], []
    start = batch_num * batch_size
    batch_data = tabfact['validation'][start: start+batch_size]
    for i in range(batch_size):
        prompt, label = process_input(sample=dict({key: value[i] for key, value in batch_data.items()}), instruction=instruction)
        batch_prompt.append(prompt)
        ground.append(str(label))
        # call llm
    batch_pred = list(map(lambda x: x.content, model.batch(batch_prompt)))
        
        #do evaluation
    accuracy = eval_fv_match(batch_pred, ground)
    print(accuracy)
        
        
        
    

        
        

0.6875


In [42]:
def display_wrong_result(pred_list, gold_list, dataset):
    for ind, (pred, gold) in enumerate(zip(pred_list, gold_list)):
        pred, gold = extract_yes_no_and_map(pred), extract_yes_no_and_map(gold)
        if pred != gold:
            sample= dataset[ind]
            df = pd.DataFrame(columns=sample["table"]["header"])
            for i, line in enumerate(sample['table']['rows']):
                df.loc[i] = line
            print(sample['statement'], sample['label'])
            print(tabulate(df, headers=df.columns, tablefmt='psql'))
        
print(display_wrong_result(batch_pred, ground, tabfact['validation']))

süper lig be the most common league to win a round in the turkish cup 1
+----+---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------+
|    | round         |   clubs remaining |   clubs involved | winners from previous round   | new entries this round   | leagues entering at this round                     |
|----+---------------+-------------------+------------------+-------------------------------+--------------------------+----------------------------------------------------|
|  0 | first round   |               156 |               86 | none                          | 86                       | tff third league & turkish regional amateur league |
|  1 | second round  |               113 |              108 | 43                            | 65                       | süper lig & tff first league & tff second league   |
|  2 | third round   |                59 |               5

In [40]:
for ind, i in enumerate(zip(batch_pred, ground)):
    print(ind, i)

0 ('1', '1')
1 ('1', '1')
2 ('1', '1')
3 ('1', '1')
4 ('1', '1')
5 ('1', '0')
6 ('0', '0')
7 ('0', '0')
8 ('0', '0')
9 ('0', '0')
10 ('1', '1')
11 ('0', '1')
12 ('1', '1')
13 ('1', '1')
14 ('0', '1')
15 ('0', '0')
16 ('0', '0')
17 ('0', '0')
18 ('1', '0')
19 ('1', '0')
20 ('The answer is 1.', '1')
21 ('0', '1')
22 ('1', '1')
23 ('0', '1')
24 ('1', '1')
25 ('0', '0')
26 ('0', '0')
27 ('1', '0')
28 ('0', '0')
29 ('0', '0')
30 ('1', '1')
31 ('1', '1')


## Answer output

In [6]:

from data_loader import TableFormat
import pandas as pd
from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.messages import HumanMessage
from langchain.prompts.prompt import PromptTemplate
from data_loader import TableLoader

table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
from utils import normalize_schema
model = ChatOpenAI(model_name="gpt-3.5-turbo-0125", openai_api_base="https://api.chatanywhere.tech/v1", openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW")
normalized_sample = table_loader.normalize_table(
                    table_loader.dataset[3])
formatter = TableFormat(format='none', data=normalized_sample)
# schema_information = pd.read_csv(f"result/aug/tabfact_test_schema.csv", index_col='table_id')
# formatter.data = normalize_schema(formatter.data, schema_information.loc[normalized_sample['id']]['schema'])
pre_instruction = PromptTemplate(input_variables=["table"], template=
"""
Below is a subtable with rows sampled, you are required to infer the data distribution and format from the sample data.
Refine commonalities about the structure within each table column.
You need to output in the following format: 
number. Column_name: Commonalities
#example format
1. championship: Names of golf tournaments are listed with some additional information (e.g., 's open, classic)

sub-table: {table}
""")
# 
output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html()) )])



In [7]:
print(output.content)

1. position: Integer values representing the team's position in the table
2. team: Names of football teams
3. played: Number of games played by each team
4. drawn: Number of games drawn by each team
5. lost: Number of games lost by each team
6. goals_for: Number of goals scored by each team
7. goals_against: Number of goals conceded by each team
8. goal_difference: Goal difference calculated as goals_for - goals_against
9. points_1: Total points earned by each team in the league


In [39]:
print(output.content)

1. game column: It appears to represent the game number, with each entry being a numerical value.
2. date column: It represents the date of the game, with each entry being in a specific date format (e.g., "month day").
3. team column: It represents the name of the team, with each entry being a text string.
4. score column: It represents the game score, with each entry showing the score in a specific format (e.g., "l 90 - 115 (ot)").
5. high_points column: It represents the player with the highest points in the game, with each entry showing the player's name and the number of points.
6. high_rebounds column: It represents the player with the highest rebounds in the game, with each entry showing the player's name and the number of rebounds.
7. high_assists column: It represents the player with the highest assists in the game, with each entry showing the player's name and the number of assists.
8. location_attendance column: It represents the location of the game and the attendance, with 

In [8]:
from data_loader import TableLoader, TableFormat
inputs = []
row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        'claim': normalized_sample['query'],
                                        # 'claim':"who is the winner of the lifetime achievement award after 2005?",
                                        'aug':  output.content
                                        }))

In [11]:
from data_loader import TableLoader, TableFormat
inputs = []
row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that provide clues for query.
sub-table: {table}
Query: verify whether the provided claim/query are true or false. {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        'claim': normalized_sample['query'],
                                        # 'claim':"who is the winner of the lifetime achievement award after 2005?",
                                        'aug':  output.content
                                        }))
# verify whether the provided claim/query are true or false.

In [35]:
from data_loader import TableLoader, TableFormat
inputs = []

row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information and data in subtable, write 3 SELECT SQL statements using table DF that complete or related to query.
The SQL may select different useful parts for the query.
sub-table: {table}
Query: {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        'claim': normalized_sample['query'] + ". The query is about 2005 Milwaukee Brewers season red uniforms vs Padres schedule",
                                        'aug':  output.content
                                        }))

In [9]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-kxgtm71G6zwC44lglIF5CfiEVVzjjc39TOtppkNAwrVA2fUW", temperature=0.1)
llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=True)
batch_pred = llm_chain.batch(inputs, return_only_outputs=True)



> Entering new LLMChain chain...
Prompt after formatting:

Our ultimate goal is to answer query based on the table. Below is a subtable with columns filtered, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the augmentation information, write a SQLITE3 SELECT SQL statement using table DF that complete query.
sub-table: <table>
<caption>1986 - 87 north west counties football league</caption>
<thead>
<tr><th>  position</th><th>               team</th><th>  played</th><th>  drawn</th><th>  lost</th><th>  goals_for</th><th>  goals_against</th><th>  goal_difference</th><th>  points_1</th></tr>
</thead>
<tbody>
<tr><td>12        </td><td>ashton town        </td><td>24      </td><td>9      </td><td>11    </td><td>29         </td><td>39             </td><td>10               </td><td>15 2      </td></tr>
<tr><td>10        </td><td>padiham            </td><td>24      </td><td>8      </td><td>12    </td><td>

In [10]:
print(batch_pred[0]['text'])

SELECT team
FROM DF
WHERE goals_for = (SELECT MAX(goals_for) FROM DF)
AND caption = '1986 - 87 north west counties football league'
AND team = 'flixton';


In [19]:
from data_loader import TableLoader, TableFormat
inputs = []

pre_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is relevant information regrading the table stucture. Carefully analyze the query, based on the augmentation information you are required to infer the data distribution and format. Write an SELECT SQL statement using table DF that is most likely to provide useful information to the query.
Query: {claim}
Extra information: {aug}
SQL: """)
inputs.append(dict({
                                        'claim': normalized_sample['query'],
                                        'aug':  output.content
                                        }))

In [22]:
from data_loader import TableLoader, TableFormat
inputs = []

row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""
Our ultimate goal is to answer query based on the table. Below is an initial SQL statement about the query. Now you can see some smaple rows from the table, improve the sql statement so that the SQL can answer the query.
initial SQL: {initial}
sub-table: {table}
Query: {claim}

SQL: """)
inputs.append(dict({"initial" : initial[0]['text'],
                                        'claim': normalized_sample['query'],
                'table': formatter.format_html(table_caption=normalized_sample['table']['caption']),
                                        }))

In [18]:
from data_loader import TableFormat
from executor import SQLManager
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
i = 0
# SQL = manager.format_sql(preds[i]['pred'].split(':')[1])


def show_table(data):
    formatter = TableFormat(format='none', data=data)
    columns = [c.strip() for c in preds[i]['pred'].split(',')]
    print(data['statement'])
    formatter.data = formatter.data.loc[:, columns]
    
    print(formatter.format_html())
    print(formatter.format_psql())
    print(data['label'])
    # data.columns = [manager.normalize_col_name(c) for c in formatter.all_data.columns]
    # data.to_sql('DF', manager.engine, if_exists='replace', index=False)
    
    # subtable = pd.read_sql(command, self.engine)
    # test_df = manager.execute_from_df(SQL, formatter.all_data)
    # return test_df
test_df = show_table(table_loader.dataset[i])
print(test_df)

the 31997 mintage be release in 2001
<table>
<thead>
<tr><th>  year</th><th>                  mintage</th></tr>
</thead>
<tbody>
<tr><td>2003  </td><td>included in hmcs bras dor</td></tr>
<tr><td>2003  </td><td>31997                    </td></tr>
<tr><td>2000  </td><td>44367                    </td></tr>
</tbody>
</table>
+--------+----------------------------+-------------------+-----------------------------+---------------+
|   year | theme                      | artist            | mintage                     |   issue price |
|--------+----------------------------+-------------------+-----------------------------+---------------|
|   2000 | steam buggy                | john mardon       | 44367                       |         59.95 |
|   2000 | the bluenose               | j franklin wright | included in steam buggy     |         59.95 |
|   2000 | the toronto                | john mardon       | included in steam buggy     |         59.95 |
|   2001 | the russell light four     | 

In [ ]:
# Answer output
pre_instruction = PromptTemplate(input_variables=["table", "claim"], template=
"""
Our ultimate goal is to answer query based on the table. Below is a SQL statement which reports error or returns no result. You need to understand the logic behind the SQL filtering and rewrite SQL to guarantee the SQL return useful information. 
sub-table: {table}
SQL Excuted: 
```{SQL}```
Query: {claim}
""")
output = model.invoke([HumanMessage(content=pre_instruction.format(table=formatter.format_html(), claim=test_sample['statement']))])

## Evaluat from local result

In [1]:
import json
from data_loader import TableLoader
table_loader = TableLoader(table_name='tabfact', split='test', small_test=True, use_sample=False)
preds = []
with open('./result/answer/tabfact_test_04-20_02-06-16.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        preds.append(json.loads(l)['pred'])

print(len(preds))
SQLs = []
with open('./result/SQL/tabfact_test_04-20_02-04-20.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        SQLs.append(json.loads(l)['pred'])
print(len(SQLs))
# do evaluation
# accuracy = eval_fv_match(pred_label, labels)



255
255


In [3]:
from utils import eval_tabfact
import json
from data_loader import TableLoader
import numpy as np
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
labels = [table_loader.normalize_table(line)['label'] for line in table_loader.dataset][:255]
answers = eval_tabfact('./result/answer/tabfact_test_04-20_02-06-16.json', labels, verbose=True)
print(answers)

['0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '1', '0', '0', '0', '0', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '1', '0', '1', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '1', '0', '0', '1', '0', '0', '1', '0', '0', '0', '0', '1', '1', '0', '1', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '1', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '1', '0', '1', '1', '0', '0', '0', '0', '1', '0', '1',

In [4]:
from utils import eval_tabfact
import json
from data_loader import TableLoader
import numpy as np
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=True, small_test=True)
labels = [table_loader.normalize_table(line)['label'] for line in table_loader.dataset]
answers = eval_tabfact('./result/answer/tabfact_test_04-18_15-20-27.json', labels, verbose=True)
print(np.where(np.array(labels) != np.array(answers)))

['0', '0', '0', '0', '1', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '0', '0', '1', '0', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '1', '1', '1', '0']
(array([ 2,  3, 14, 21, 25, 27, 32, 35]),)


In [39]:
data = table_loader.dataset[i]
df = pd.DataFrame(columns=data["table"]["header"])
for i, line in enumerate(data['table']['rows']):
    df.loc[i] = line
data = df

In [40]:
formatter = TableFormat(format='none', data=table_loader.dataset[i])
from functools import partial
data_func = partial(pd.to_datetime, dayfirst=True, format='mixed')
formatter.all_data['date'] = data_func(formatter.all_data['date'])
formatter.all_data['date'] = pd.to_datetime(formatter.all_data['date'], format='%Y-%m')
formatter.all_data['date'] = formatter.all_data['date'].dt.strftime('%Y-%m-%d')

In [72]:
import pandas as pd
formatter = TableFormat(format='none', data=table_loader.dataset[i])
schema_information = pd.read_csv(f"result/aug/tabfact_test_schema.csv", index_col='table_id')
formatter.normalize_schema(schema_information.loc[table_loader.dataset[i]['table']['id']]['schema'])
formatter.all_data.to_sql('DF', con=engine, if_exists='replace', index=False)

27

In [2]:
inds = [0,   1,  11,  14,  22,  24,  26,  31,  37,  38,  39,  40,  47,
       49,  50,  59,  62,  63,  65,  66,  68,  69,  70,  75,  80,  81,
       82,  90,  91,  93,  95,  99, 100, 105, 106, 108, 110, 116, 119,
124, 129, 136, 137, 145, 147, 148, 151, 153, 155, 158, 160, 164,
167, 169, 171, 173, 175, 179, 180, 181, 187, 188, 190, 192, 198,
200, 207, 209, 210, 211, 217, 225, 227, 230, 233, 236, 237, 241,
251]
len(inds)

79

In [3]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
# table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
i = 25

def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['table']['id'])
    print(data['statement'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])
print(table_loader.dataset[i]['table']['caption'])
show_table(table_loader.dataset[i]) 

1989 u.s. open (golf)
2-17231086-6.html.csv
3 of the people tie for fifth place be from the united state
+---------+---------------------+---------------+-------------------------+----------+---------+
| place   | player              | country       | score                   | to_par   |   money |
|---------+---------------------+---------------+-------------------------+----------+---------|
| 1       | curtis strange      | united states | 71 + 64 + 73 + 70 = 278 | - 2      |  200000 |
| t2      | chip beck           | united states | 71 + 69 + 71 + 68 = 279 | - 1      |   67823 |
| t2      | mark mccumber       | united states | 70 + 68 + 72 + 69 = 279 | - 1      |   67823 |
| t2      | ian woosnam         | wales         | 70 + 68 + 73 + 68 = 279 | - 1      |   67823 |
| 5       | brian claar         | united states | 71 + 72 + 68 + 69 = 280 | e        |   34345 |
| t6      | masashi ozaki       | japan         | 70 + 71 + 68 + 72 = 281 | + 1      |   28220 |
| t6      | scott simp

#### show wikitable

In [42]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='wikitable', split='validation', use_sample=True, small_test=False)
# table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
i = 1


def show_table(data, execute=False):
    formatter = TableFormat(format='none', data=data)
    print(data['id'])
    print(data['query'])
    print(formatter.format_psql())
    # print(preds[i])
    # print(SQLs[i])
    # test_df = manager.execute_from_df(SQLs[i], formatter.all_data)
    # print(test_df)
    print(data['label'])

show_table(table_loader.normalize_table(table_loader.dataset[i]) )

nt-11806
what was the time difference between the first place finisher and the eighth place finisher?
+--------+-----------------------+---------------+---------+
| Rank   | Athlete               | Nationality   | Time    |
|--------+-----------------------+---------------+---------|
|        | Liliana Barbulescu    | Romania       | 2:00.06 |
|        | Anna Zagórska         | Poland        | 2:00.11 |
|        | Irina Vashentseva     | Russia        | 2:00.77 |
| 4      | Laura Gerber          | Switzerland   | 2:01.39 |
| 5      | Christiane dos Santos | Brazil        | 2:01.53 |
| 6      | Tamara Volkova        | Ukraine       | 2:01.86 |
| 7      | Tatyana Yegorova      | Russia        | 2:02.64 |
| 8      | Sandra Teixeira       | Portugal      | 2:03.01 |
+--------+-----------------------+---------------+---------+
['2.95']


In [40]:
for ind, i in enumerate(table_loader.dataset['answers']):
    print(ind, i)

0 ['Dmitry Mikhailovich Golitsyn']
1 ['Kyunghyun Kim', 'Christoffer Lindhe', 'Arnost Petracek', 'Ronystony Cordeiro da Silva', 'Grant Patterson', 'Arnulfo Castorena']
2 ['Italy']
3 ['20']
4 ['"Never Tell"']
5 ['2']
6 ['Young Forever']
7 ['4']
8 ['2:07:02.8248']
9 ['Zhu Ting']
10 ['21']
11 ['Canada']
12 ['9']
13 ['no']
14 ['21']
15 ['Burton Township, Ohio and Troy Township, Ohio']
16 ['6']
17 ['below']
18 ['14']
19 ['13']
20 ['Allianz Riviera']
21 ['10']
22 ['2001']
23 ['4']
24 ['Sopwith Triplane s/n N5460']
25 ['Mr B. Owen- Jones']
26 ['9']
27 ['7']
28 ['Ryan Hunter-Reay']
29 ['4']
30 ['60.28 m']
31 ['6']
32 ['2007']
33 ['170 cm (5 ft 7 in)']
34 ['2']
35 ['14']
36 ['1982-1985']
37 ['12']
38 ['31']
39 ['473,000']
40 ['8']
41 ['2012']
42 ['Holon']
43 ['Addis Ababa']
44 ['.900 silver']
45 ['Yugoslavia']
46 ['7']
47 ['Chris Van Hollen']
48 ['Khaled Yousef Al-Obaidli']
49 ['2']
50 ['Kodachrome 40 film']
51 ['Paul Christy']
52 ['yes']
53 ['6']
54 ['Ramsay Carelse']
55 ['1943/44']
56 ['4']
57

In [4]:
from executor import SQLManager
from sqlalchemy import create_engine
from data_loader import TableLoader, TableFormat
table_loader = TableLoader(table_name='sqa', split='test', use_sample=True)
i = 124
def show_table(data):
    formatter = TableFormat(format='none', data=data)
    print(data['id'])
    print(data['query'])
    print(formatter.format_psql())
    print(data['label'])
    # sum, ope = split_answer(data[ind]['predict'])
    # print(ope)
    # print(manager.format_sql(ope, table_name='DF'))
show_table(table_loader.normalize_table(table_loader.dataset[i])) 
print(all_sql[i])
print(all_data[i])

nu-108
what are all of the years? what are their national cup notes?
+---------+------------+----------+------------------------+-----------------------+----------------+
| Year    |   Division | League   | Reg_Season             | Playoffs              | National_Cup   |
|---------+------------+----------+------------------------+-----------------------+----------------|
| 1934/35 |        nan | ASL      | 6th                    | No playoff            | ?              |
| 1935/36 |        nan | ASL      | 2nd                    | No playoff            | ?              |
| 1936/37 |        nan | ASL      | 2nd, American          | 1st Round             | ?              |
| 1937/38 |        nan | ASL      | 4th, American          | Did not qualify       | ?              |
| 1938/39 |        nan | ASL      | 5th, American          | Did not qualify       | ?              |
| 1939/40 |        nan | ASL      | 2nd(t)                 | No playoff            | Co-champion    |
| 1940/41 |  

NameError: name 'all_sql' is not defined

In [12]:
# show table query
#debugger
from data_loader import TableFormat, TableLoader
from executor import SQLManager
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine)
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True)
def show_table(data):
    formatter = TableFormat(format='none', data=data)
    print(data['statement'])
    print(data['table']['caption'])
    # print(formatter.format_html())
    print(formatter.format_psql())
    print(data['label'])
    # data.columns = [manager.normalize_col_name(c) for c in formatter.all_data.columns]
    # data.to_sql('DF', manager.engine, if_exists='replace', index=False)
    
    # subtable = pd.read_sql(command, self.engine)
    # test_df = manager.execute_from_df(SQL, formatter.all_data)
    # return test_df
# test_df = show_table(table_loader.dataset[i])
show_table(table_loader.dataset[130])
for ind, i in enumerate(table_loader.dataset['statement']):
    print(ind, i)
 

the country of ecuador be the only country that use the us dollar (usd) as its currency , and the central bank be the federal reserve
economy of south america
+-----------+---------------------------------+--------------+-------------+------------------------------+
| country   | currency                        |   c_1_euro_= |   c_1_usd_= | central_bank                 |
|-----------+---------------------------------+--------------+-------------+------------------------------|
| argentina | argentine peso (ars)            |      5.65    |      4.2    | central bank of argentina    |
| bolivia   | bolivian boliviano (bob)        |     11.0985  |      7.5708 | central bank of bolivia      |
| brazil    | brazilian real (brl)            |      2.58963 |      1.7665 | central bank of brazil       |
| chile     | chilean peso (clp)              |    701.02    |    507.58   | central bank of chile        |
| colombia  | colombian peso (cop)            |   2593.2     |   1885.74   | bank of 

### Evaluate SQA


In [9]:
import json
all_data = []
all_sql = []
with open('./result/answer/sqa_test_04-15_11-44-40.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        data = json.loads(l)
        all_data.append(data)
print(len(all_data))
with open('./result/SQL/sqa_test_04-15_10-35-18.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        data = json.loads(l)
        all_sql.append(data)
print(len(all_sql))
        

10
10


### Evaluate WikiTable

In [1]:
import re
import string
import recognizers_suite
import recognizers_suite as Recognizers
from recognizers_text import Culture, ModelResult
def str_normalize(user_input, recognition_types=None):
    """A string normalizer which recognize and normalize value based on recognizers_suite"""
    user_input = str(user_input)
    user_input = user_input.replace("\\n", "; ")

    def replace_by_idx_pairs(orig_str, strs_to_replace, idx_pairs):
        assert len(strs_to_replace) == len(idx_pairs)
        last_end = 0
        to_concat = []
        for idx_pair, str_to_replace in zip(idx_pairs, strs_to_replace):
            to_concat.append(orig_str[last_end : idx_pair[0]])
            to_concat.append(str_to_replace)
            last_end = idx_pair[1]
        to_concat.append(orig_str[last_end:])
        return ''.join(to_concat)

    if recognition_types is None:
        recognition_types = [
            "datetime",
            "number",
            "ordinal",
            "percentage",
            "age",
            "currency",
            "dimension",
            "temperature",
        ]
    culture = Culture.English
    for recognition_type in recognition_types:
        if re.match("\d+/\d+", user_input):
            # avoid calculating str as 1991/92
            continue
        recognized_list = getattr(
            recognizers_suite, "recognize_{}".format(recognition_type)
        )(
            user_input, culture
        )  # may match multiple parts
        strs_to_replace = []
        idx_pairs = []
        for recognized in recognized_list:
            if not recognition_type == 'datetime':
                recognized_value = recognized.resolution['value']
                if str(recognized_value).startswith("P"):
                    # if the datetime is a period:
                    continue
                else:
                    strs_to_replace.append(recognized_value)
                    idx_pairs.append((recognized.start, recognized.end + 1))
            else:
                if recognized.resolution:  # in some cases, this variable could be none.
                    if len(recognized.resolution['values']) == 1:
                        strs_to_replace.append(
                            recognized.resolution['values'][0]['timex']
                        )  # We use timex as normalization
                        idx_pairs.append((recognized.start, recognized.end + 1))

        if len(strs_to_replace) > 0:
            user_input = replace_by_idx_pairs(user_input, strs_to_replace, idx_pairs)

    if re.match("(.*)-(.*)-(.*) 00:00:00", user_input):
        user_input = user_input[: -len("00:00:00") - 1]
        # '2008-04-13 00:00:00' -> '2008-04-13'
    return user_input


def normalize_answer(s):
    def remove_articles(text):
        return re.sub(re.compile(r"\b(a|an|the)\b", re.UNICODE), " ", text)

    def whilt_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return whilt_space_fix(remove_articles(remove_punc(lower(s))))


In [2]:

import sys, os, re, argparse
import unicodedata
from codecs import open
from math import isnan, isinf
from abc import ABCMeta, abstractmethod


################ String Normalization ################

def normalize(x):
    if not isinstance(x, str):
        x = x.decode('utf8', errors='ignore')
    # Remove diacritics
    x = ''.join(c for c in unicodedata.normalize('NFKD', x)
                if unicodedata.category(c) != 'Mn')
    # Normalize quotes and dashes
    x = re.sub(r"[‘’´`]", "'", x)
    x = re.sub(r"[“”]", "\"", x)
    x = re.sub(r"[‐‑‒–—−]", "-", x)
    while True:
        old_x = x
        # Remove citations
        x = re.sub(r"((?<!^)\[[^\]]*\]|\[\d+\]|[•♦†‡*#+])*$", "", x.strip())
        # Remove details in parenthesis
        x = re.sub(r"(?<!^)( \([^)]*\))*$", "", x.strip())
        # Remove outermost quotation mark
        x = re.sub(r'^"([^"]*)"$', r'\1', x.strip())
        if x == old_x:
            break
    # Remove final '.'
    if x and x[-1] == '.':
        x = x[:-1]
    # Collapse whitespaces and convert to lower case
    x = re.sub(r'\s+', ' ', x, flags=re.U).lower().strip()
    return x


################ Value Types ################

class Value(object):
    __metaclass__ = ABCMeta

    # Should be populated with the normalized string
    _normalized = None

    @abstractmethod
    def match(self, other):
        """Return True if the value matches the other value.
        Args:
            other (Value)
        Returns:
            a boolean
        """
        pass

    @property
    def normalized(self):
        return self._normalized


class StringValue(Value):

    def __init__(self, content):
        assert isinstance(content, str)
        self._normalized = normalize(content)
        self._hash = hash(self._normalized)

    def __eq__(self, other):
        return isinstance(other, StringValue) and self.normalized == other.normalized

    def __hash__(self):
        return self._hash

    def __str__(self):
        return 'S' + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        return self.normalized == other.normalized


class NumberValue(Value):

    def __init__(self, amount, original_string=None):
        assert isinstance(amount, (int, float))
        if abs(amount - round(amount)) < 1e-6:
            self._amount = int(amount)
        else:
            self._amount = float(amount)
        if not original_string:
            self._normalized = str(self._amount)
        else:
            self._normalized = normalize(original_string)
        self._hash = hash(self._amount)

    @property
    def amount(self):
        return self._amount

    def __eq__(self, other):
        return isinstance(other, NumberValue) and self.amount == other.amount

    def __hash__(self):
        return self._hash

    def __str__(self):
        return ('N(%f)' % self.amount) + str([self.normalized])

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, NumberValue):
            return abs(self.amount - other.amount) < 1e-6
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a number.
        Return:
            the number (int or float) if successful; otherwise None.
        """
        try:
            return int(text)
        except:
            try:
                amount = float(text)
                assert not isnan(amount) and not isinf(amount)
                return amount
            except:
                return None


class DateValue(Value):

    def __init__(self, year, month, day, original_string=None):
        """Create a new DateValue. Placeholders are marked as -1."""
        assert isinstance(year, int)
        assert isinstance(month, int) and (month == -1 or 1 <= month <= 12)
        assert isinstance(day, int) and (day == -1 or 1 <= day <= 31)
        assert not (year == month == day == -1)
        self._year = year
        self._month = month
        self._day = day
        if not original_string:
            self._normalized = '{}-{}-{}'.format(
                year if year != -1 else 'xx',
                month if month != -1 else 'xx',
                day if day != '-1' else 'xx')
        else:
            self._normalized = normalize(original_string)
        self._hash = hash((self._year, self._month, self._day))

    @property
    def ymd(self):
        return (self._year, self._month, self._day)

    def __eq__(self, other):
        return isinstance(other, DateValue) and self.ymd == other.ymd

    def __hash__(self):
        return self._hash

    def __str__(self):
        return (('D(%d,%d,%d)' % (self._year, self._month, self._day))
                + str([self._normalized]))

    __repr__ = __str__

    def match(self, other):
        assert isinstance(other, Value)
        if self.normalized == other.normalized:
            return True
        if isinstance(other, DateValue):
            return self.ymd == other.ymd
        return False

    @staticmethod
    def parse(text):
        """Try to parse into a date.
        Return:
            tuple (year, month, date) if successful; otherwise None.
        """
        try:
            ymd = text.lower().split('-')
            assert len(ymd) == 3
            year = -1 if ymd[0] in ('xx', 'xxxx') else int(ymd[0])
            month = -1 if ymd[1] == 'xx' else int(ymd[1])
            day = -1 if ymd[2] == 'xx' else int(ymd[2])
            assert not (year == month == day == -1)
            assert month == -1 or 1 <= month <= 12
            assert day == -1 or 1 <= day <= 31
            return (year, month, day)
        except:
            return None


################ Value Instantiation ################

def to_value(original_string, corenlp_value=None):
    """Convert the string to Value object.
    Args:
        original_string (basestring): Original string
        corenlp_value (basestring): Optional value returned from CoreNLP
    Returns:
        Value
    """
    if isinstance(original_string, Value):
        # Already a Value
        return original_string
    if not corenlp_value:
        corenlp_value = original_string
    # Number?
    amount = NumberValue.parse(corenlp_value)
    if amount is not None:
        return NumberValue(amount, original_string)
    # Date?
    ymd = DateValue.parse(corenlp_value)
    if ymd is not None:
        if ymd[1] == ymd[2] == -1:
            return NumberValue(ymd[0], original_string)
        else:
            return DateValue(ymd[0], ymd[1], ymd[2], original_string)
    # String.
    return StringValue(original_string)


def to_value_list(original_strings, corenlp_values=None):
    """Convert a list of strings to a list of Values
    Args:
        original_strings (list[basestring])
        corenlp_values (list[basestring or None])
    Returns:
        list[Value]
    """
    assert isinstance(original_strings, (list, tuple, set))
    if corenlp_values is not None:
        assert isinstance(corenlp_values, (list, tuple, set))
        assert len(original_strings) == len(corenlp_values)
        return list(set(to_value(x, y) for (x, y)
                        in zip(original_strings, corenlp_values)))
    else:
        return list(set(to_value(x) for x in original_strings))


################ Check the Predicted Denotations ################

def check_denotation(target_values, predicted_values):
    """Return True if the predicted denotation is correct.

    Args:
        target_values (list[Value])
        predicted_values (list[Value])
    Returns:
        bool
    """
    # Check size
    if len(target_values) != len(predicted_values):
        return False
    # Check items
    for target in target_values:
        if not any(target.match(pred) for pred in predicted_values):
            return False
    return True
def tsv_unescape(x):
    """Unescape strings in the TSV file.
    Escaped characters include:
        newline (0x10) -> backslash + n
        vertical bar (0x7C) -> backslash + p
        backslash (0x5C) -> backslash + backslash

    Args:
        x (str or unicode)
    Returns:
        a unicode
    """
    return x.replace(r'\n', '\n').replace(r'\p', '|').replace('\\\\', '\\')

def tsv_unescape_list(x):
    """Unescape a list in the TSV file.
    List items are joined with vertical bars (0x5C)

    Args:
        x (str or unicode)
    Returns:
        a list of unicodes
    """
    return [tsv_unescape(y) for y in x.split('|')]

In [3]:
tagged_dataset_path = './result/data/wikitable/tagged/data'
target_values_map = {}
for filename in os.listdir(tagged_dataset_path):
    if filename[0]=='.':
        continue
    filename = os.path.join(tagged_dataset_path, filename)
    print('Reading dataset from', filename)
    with open(filename, 'r', 'utf8') as fin:
        header = fin.readline().rstrip('\n').split('\t')
        for line in fin:
            stuff = dict(zip(header, line.rstrip('\n').split('\t')))
            ex_id = stuff['id']
            original_strings = tsv_unescape_list(stuff['targetValue'])
            canon_strings = tsv_unescape_list(stuff['targetCanon'])

            target_values_map[ex_id] = to_value_list(
                original_strings, canon_strings)

Reading dataset from ./result/data/wikitable/tagged/data/training.tagged
Reading dataset from ./result/data/wikitable/tagged/data/pristine-unseen-tables.tagged
Reading dataset from ./result/data/wikitable/tagged/data/pristine-seen-tables.tagged


##### read_csv

In [4]:
import pandas as pd
data = pd.read_csv('./result/answer/wikitable_zh_05-01_15-49-08.csv', encoding='utf-8') 

In [5]:
len(data)

50

In [20]:
data['final_answer'] = data['preds'].apply(lambda x: x.split('Answer:')[-1].strip())


In [27]:
data

,preds,statements,ids,final_answer
0,Thought: \n1. The SQL query filters the DF tab...,did dmitry mikhailovich golitsyn or andrey kir...,nu-3365,Andrey Kirillovich Razumovsky served as ambass...
1,Thought: \n1. Look at the sub-table and find t...,which person(s) had a time of at least 40 seco...,nu-3138,"Kyunghyun Kim, Christoffer Lindhe, Arnost Petr..."
2,Thought: \n1. The query is counting the number...,what country had the largest number of drivers?,nu-2135,The country with the largest number of drivers...
3,Thought: The SQL query is counting the total n...,what is the total number of competition?,nu-290,18
4,"Thought: The SQL query is selecting the ""Episo...",the title of the last show was...,nu-3940,"""Never Tell"""
5,Thought: The SQL query is selecting the count ...,what is the total number of games played again...,nu-4028,3
6,Thought: \n1. The SQL query is filtering for t...,"which episode came first, young forever or you...",nu-3762,"There are no episodes with the titles ""Young F..."
7,Thought: \n- The SQL query selects the count o...,how many top goalscorers have 30 or more goals?,nu-1889,4
8,Thought: \nTo determine how long it took Ryan ...,how long did it take ryan briscoe to finish th...,nu-336,To calculate how long it took Ryan Briscoe to ...
9,Thought: \nWe need to find the player who scor...,who is china's top scorer in friendly matches?,nu-2563,Cannot get answer from sub-table.


In [24]:
acc = 0
for index, row in data.iterrows():
    id = row['ids']
    target = target_values_map[id]
     
    preds = [row['final_answer']]
    pred_answer = to_value_list(preds)
    if check_denotation(pred_answer, target):
        acc += 1
    else:
        print(index)
print(acc)

0
1
2
3
5
6
8
9
10
12
13
14
15
16
17
19
20
21
22
25
26
30
31
32
33
34
37
38
39
41
42
47
49
17


In [31]:
culture = Culture.English
Recognizers.recognize_number('The total number of competitions is 20.', culture)[0].resolution['value']

'20'

In [ ]:
print(output.content)

The table provides information about different games played by a football team. It includes details like the week number, date of the game, opponent team, game result, game site, NFL recap, and the attendance for each game.


In [27]:
str_normalize('The total number of competitions is 20.')

'The total number of competitions is 20.'

#### decorator

In [ ]:
from enum import Enum
class AugType(Enum):
    augmentation_aug = 'aug'
    schema_aug = 'schema'


In [27]:
def my_decorator(type):
    def wrapper(func):
        print('wrapper func')
        def subwrapper(self, *args, **kwargs):
            print('subwrapper')
            self.function_mapping[type] = func
            return func(self, *args, **kwargs)
        return subwrapper
    return wrapper
class MyClass:
    def __init__(self):
        self.value = 0
        self.function_mapping = {}
    def execute_method(self, type):
       method = self.function_mapping.get(type, None)
       if method:
           method(self)
       else:
           print(f"Method {type} not found.")

    @my_decorator(type='sum')
    def summary(self):
        print("This is the summary method.")

a = MyClass()
a.summary()
a.execute_method('sum')

wrapper func
subwrapper
This is the summary method.
This is the summary method.


In [32]:
class MyClass:
   def __init__(self):
       self.function_mapping = {}
 
   def execute_method(self, type):
       method = self.function_mapping.get(type, None)
       if method:
           method()
       else:
           print(f"Method {type} not found.")
 
   def register_function(self, type):
       def decorator(func):
           self.function_mapping[type] = func
           return func
       return decorator
 
# 使用装饰器来注册函数
my_instance = MyClass()
@my_instance.register_function('sum')
def summary():
   print("This is the summary method.")
 
@my_instance.register_function('aug')
def augmentation():
   print("This is the augmentation method.")
 
@my_instance.register_function('schema')
def schema():
   print("This is the schema method.")
 
# 根据type参数调用对应的方法
my_instance.execute_method('sum')  # 输出: This is the summary method.
my_instance.execute_method('aug')  # 输出: This is the augmentation method.
my_instance.execute_method('schema')  # 输出: This is the schema method.
my_instance.execute_method('nonexistent')  # 输出: Method nonexistent not found.

This is the summary method.
This is the augmentation method.
This is the schema method.
Method nonexistent not found.


In [4]:
class BaseClass():
    def answer(self):
        return "i am baseclass"

class MetaClass(type):
    def __new__(cls, *args, **kwargs):
        return super().__new__(cls, *args, **kwargs)

# 什么是元类， 元类是创建类的类。
# 创建过程：type --> class(对象) --> 对象 
class User(BaseClass, metaclass=MetaClass):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return "user"

In [24]:
def repeat(number=3):
   def decorator(func):
       def wrapper(self, *args, **kwargs):
           print('nihao')
           for _ in range(number):
               value = func(self, *args, **kwargs)
           return value
       return wrapper
   return decorator
 
class MyClass:
   @repeat(number=5)
   def say_hello(self):
    #    print("Hello!")
        pass
 
# 创建 MyClass 的实例并调用 say_hello 方法
my_instance = MyClass()
my_instance.say_hello()

nihao


In [6]:
a.answer()

'i am baseclass'

In [7]:
class Meta(type):
    def __new__(cls, name, bases, attrs):
        for key, value in attrs.items():
            if callable(value) and not key.startswith("__"):
                attrs[key] = my_decorator(value)
        return super(Meta, cls).__new__(cls, name, bases, attrs)

class MyClass(metaclass=Meta):
    def __init__(self):
        self.value = 0

    def my_decorator(func):
        def wrapper(self, *args, **kwargs):
            print("Before function call")
            result = func(self, *args, **kwargs)
            print("After function call")
            return result
        return wrapper

    def my_method(self, x):
        self.value += x
        print(f"Value is now {self.value}")

NameError: name 'my_decorator' is not defined